In [1]:
import pandas as pd
import numpy as np
import time
import gc

In [ ]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train_sample = pd.read_csv(path + 'train_sample_cleaned.csv')

In [ ]:
train_sample.dtypes

# explore time

In [ ]:
print('training')
print(train_sample.timestamp.max() - train_sample.timestamp.min())
print('max: {}'.format(train_sample.timestamp.max()))
print('min: {}'.format(train_sample.timestamp.min()))
print('==========')
print('testing')
print(train_sample.timestamp.max() - train_sample.timestamp.min())
print('max: {}'.format(train_sample.timestamp.max()))
print('min: {}'.format(train_sample.timestamp.min()))

# Use 10% train as Sample

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned.csv')
test = pd.read_csv(path + 'test_cleaned.csv')

ratio = 0.1
length = int(len(train) * ratio)
train = train[-length:None]

# del train
gc.collect()

0

# Get K Fold

In [7]:
from sklearn.model_selection import KFold
K = 6
kf = KFold(n_splits=K, shuffle = False)

history_index = []
train_index = []
for h,t in kf.split(train):
    history_index.append(h)
    train_index.append(t)

In [46]:
df_history = train.iloc[history_index[-1]].copy()
df_train = train.iloc[train_index[-1]].copy()

import sys
print(sys.getsizeof(df_train)/ 1024 **3)
print(len(df_train))

0.3444103002548218
3081731


# Get Group

In [22]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',
              'timestamp',
              'minute',
              'second']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(max(train[col].max(),test[col].max() ) + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the combination will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

# Combine Features

In [24]:
from itertools import combinations


combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel', 
              'day',
              'hour',
              'minute',
              'second']

def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__

counter = 0
for cols in combinations(combine_col, 2):
    counter += 1
print(counter)

36


In [26]:
combines = []
for comb in combinations(combine_col, 2):
    combines.append(comb)

## count
plan 1. count from historical data  
plan 2. count from all data

In [121]:
def count(df_history, df_train, cols):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)

cols = combines[1]
feature_name = col_name(cols, func=count)
print(feature_name)
df_train[feature_name] = count(df_history, df_train, cols)

ip_device_count


# mean
mean P(target | feature combination)

In [ ]:
purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer))
Get the conditional Probability only from historical data and apply to train data.

P(replay | X feature combination) = P( replay & X feature combination) / P (X feature combination)
=(count(replay & X feature combination) / count(total)) / (count(X feature combination) / count(total))
= count(replay & X feature combination) / count(X feature combination)
= sum((replay & X feature combination)) / count(X feature combination)
= sum((replay or not replayed & X feature combination)) / count(X feature combination)# since replay is 1, not replay is 0
= sum( X feature combination) / count(X feature combination)
= mean(X feature combination)

In [120]:
def mean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    mean_map = df_history.groupby(group_history)[target].mean()
    # mean_map: key - encoding, value - target mean
    return group.map(mean_map).fillna(-1)

cols = combines[1]
feature_name = col_name(cols, func=mean)
print(feature_name)
df_train[feature_name] = mean(df_history, df_train, cols, target='is_attributed')

ip_device_mean


# reversemean
reverse mean P(feature combination | target)

In [118]:
def rversemean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    positive = group_history[df_history[target] == 1]
    negative = group_history[df_history[target] == 0]
    index_p = set(positive.unique())
    index_n = set(negative.unique())
    index_n.difference_update(index_p)
    map_reverse_p = positive.groupby(positive).count() / len(positive)
    map_reverse_n = pd.Series(np.zeros(len(index_n)), index=index_n)
    map_reverse = pd.concat([map_reverse_p, map_reverse_n])
    return group.map(map_reverse).fillna(-1)

cols = combines[1]
feature_name = col_name(cols, func=rversemean)
print(feature_name)
df_train[feature_name] = rversemean(df_history, df_train, cols, target='is_attributed')

ip_device_rversemean


In [122]:
df_train


,ip,app,device,os,channel,year,month,week,day,hour,timestamp,minute,second,is_attributed,ip_device_count,ip_device_rversemean,ip_device_mean
181822159,60051,18,1,19,121,2017,11,45,9,14,1510239536,58,56,0,1077.0,0.000000,0.000000
181822160,114107,3,1,13,135,2017,11,45,9,14,1510239536,58,56,0,1379.0,0.000026,0.000725
181822161,66000,9,1,8,127,2017,11,45,9,14,1510239536,58,56,0,16386.0,0.000000,0.000000
181822162,73516,9,1,10,466,2017,11,45,9,14,1510239536,58,56,0,61453.0,0.002683,0.001676
181822163,42561,18,1,47,107,2017,11,45,9,14,1510239536,58,56,0,417.0,0.000000,0.000000
181822164,119369,7,1,19,101,2017,11,45,9,14,1510239536,58,56,0,31757.0,0.000156,0.000189
181822165,95766,9,1,13,127,2017,11,45,9,14,1510239536,58,56,0,34223.0,0.000156,0.000175
181822166,82816,18,1,18,107,2017,11,45,9,14,1510239536,58,56,0,3361.0,0.000026,0.000298
181822167,77295,9,1,10,127,2017,11,45,9,14,1510239536,58,56,0,1034.0,0.000026,0.000967
181822168,37428,26,1,13,266,2017,11,45,9,14,1510239536,58,56,0,451.0,0.000000,0.000000


In [75]:
target='is_attributed'
mean_map_reverse = group_history.groupby(df_history[target])

In [97]:
positive = group_history[df_history[target] == 1]
posi_group = positive.groupby(positive).count() / len(positive)

In [99]:
negative = group_history[df_history[target] == 0]

In [116]:
# encoding df's cols into a new series
group = get_group(df_train, cols)
# encoding df_history's cols into a new series
group_history = get_group(df_history, cols)
# get the conditional probability p(target| feature combination. eg, artist_name_composer) 
positive = group_history[df_history[target] == 1]
negative = group_history[df_history[target] == 0]
index_p = set(positive.unique())
index_n = set(negative.unique())
index_n.difference_update(index_p)
values = np.zeros(len(index_n))
negative_map = pd.Series(values, index=index_n)
map_reverse_p = positive.groupby(positive).count() / len(positive)
map_reverse = pd.concat([map_reverse_p, negative_map])

In [117]:
map_reverse

10001         0.000130
270001        0.000052
310000        0.000026
630001        0.000026
1070001       0.000026
1960001       0.000026
2060000       0.000026
2220251       0.000026
2260000       0.000026
2680000       0.000026
2840000       0.000026
3150001       0.000026
3530001       0.000026
4690000       0.000026
5010001       0.000026
5190001       0.000026
5220001       0.000026
5730001       0.000026
5730166       0.000026
5960000       0.000026
5961026       0.000026
6250001       0.000052
6580030       0.000026
6790001       0.000026
6810000       0.000026
6990001       0.000026
7070001       0.000026
7240000       0.000026
7280001       0.000026
7380000       0.000026
                ...   
3458990001    0.000000
2967470001    0.000000
3458990002    0.000000
247463866     0.000000
3619160001    0.000000
3127640002    0.000000
3127640001    0.000000
833880005     0.000000
833880002     0.000000
14680002      0.000000
3455320002    0.000000
838860745     0.000000
666370001  

In [109]:
len(map_reverse.index.values)

175128

In [110]:
len(set(map_reverse.index.values))

153526